# Poseidon TCP Conflict Study Advanced Topology

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for performing iperf3 tests on short rtt fat pipe networks.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [FABlib API](https://learn.fabric-testbed.net/docs/fablib/fablib.html)

## Basic FABRIC Slice Configuration

In [1]:
import json
import traceback
from time import sleep
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

In [2]:
try:
    fablib.list_sites(fields=["name", "cores_available", "ram_available", "nic_basic_available", "nic_connectx_5_available", "nic_connectx_6_available"])
    #fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

Name,Cores Available,RAM Available,Basic NIC Available,ConnectX-5 Available,ConnectX-6 Available
STAR,380,3056,759,5,2
TACC,220,2168,621,4,2
NEWY,128,1024,254,3,3
GPN,266,2320,609,2,2
MICH,190,1528,381,1,2
DALL,152,1328,363,1,2
SALT,182,1496,379,1,1
INDI,192,1536,381,2,2
FIU,280,2408,632,4,2
CERN,134,1612,750,4,6


## Configure Slice Parameters

In [3]:
# Slice 
slice_name = 'PoSeiDon-TCP-Experiment-ConnectX5_6'

# Routers
r1_name = "poseidon-router-1"
r2_name = "poseidon-router-2"

net_r1_r2_name = "net_r1_r2"
subnet_r1_r2 = "192.168.1.0/30"

router_cores = 24
router_ram = 32
router_disk = 100

# Hosts
h1_name = "poseidon-sender-1"
h2_name = "poseidon-sender-2"

net_h1_r1_name = 'net_h1_r1'
subnet_h1_r1 = "192.168.2.0/30"

net_h2_r1_name = 'net_h2_r1'
subnet_h2_r1 = "192.168.3.0/30"

h3_name = "poseidon-receiver-1"
h4_name = "poseidon-receiver-2"

net_h3_r2_name = 'net_h3_r2'
subnet_h3_r2 = "192.168.4.0/30"

net_h4_r2_name = 'net_h4_r2'
subnet_h4_r2 = "192.168.5.0/30"

host_cores = 26
host_ram = 32
host_disk = 100

# Sites TACC STAR NCSA MICH DALL MAX MASS WASH CLEM SALT UTAH UCSD FIU
r1_site = 'WASH'
r2_site = 'NCSA'
h1_site = 'CLEM' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR
h2_site = 'CLEM' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR
h3_site = 'TACC' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR
h4_site = 'TACC' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR

# All node properties
#username = 'ubuntu'
os_image = 'default_ubuntu_20'

### Create the Slice

In [4]:
try:
    #Create Slice
    fabric_slice = fablib.new_slice(name=slice_name)
    
    # Add poseidon r1 node
    r1 = fabric_slice.add_node(
                    name=r1_name,
                    site=r1_site,
                    image=os_image,
                    cores=router_cores,
                    ram=router_ram,
                    disk=router_disk)
    
    r1_h1_h2_nic = r1.add_component(model='NIC_ConnectX_6', name="r1_h1_h2_nic")
    r1_iface_to_h1 = r1_h1_h2_nic.get_interfaces()[0]
    
    r1_iface_to_h2 = r1_h1_h2_nic.get_interfaces()[1]
    
    r1_r2_nic = r1.add_component(model='NIC_ConnectX_6', name="r1_r2_nic")
    r1_iface_to_r2 = r1_r2_nic.get_interfaces()[0]
    
    # Add poseidon r2 node
    r2 = fabric_slice.add_node(
                    name=r2_name,
                    site=r2_site,
                    image=os_image,
                    cores=router_cores,
                    ram=router_ram,
                    disk=router_disk)
    
    r2_h3_h4_nic = r2.add_component(model='NIC_ConnectX_6', name="r2_h3_h4_nic")
    r2_iface_to_h3 = r2_h3_h4_nic.get_interfaces()[0]
    
    r2_iface_to_h4 = r2_h3_h4_nic.get_interfaces()[1]
    
    r2_r1_nic = r2.add_component(model='NIC_ConnectX_6', name="r2_r1_nic")
    r2_iface_to_r1 = r2_r1_nic.get_interfaces()[0]

    # Add poseidon host node h1
    h1 = fabric_slice.add_node(
                    name=h1_name,
                    site=h1_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h1_iface = h1.add_component(model='NIC_ConnectX_6', name="h1_nic").get_interfaces()[0]
    
    # Add poseidon host node h2
    h2 = fabric_slice.add_node(
                    name=h2_name,
                    site=h2_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h2_iface = h2.add_component(model='NIC_ConnectX_6', name="h2_nic").get_interfaces()[0]
    
    # Add poseidon host node h3
    h3 = fabric_slice.add_node(
                    name=h3_name,
                    site=h3_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h3_iface = h3.add_component(model='NIC_ConnectX_6', name="h3_nic").get_interfaces()[0]
    
    # Add poseidon host node h4
    h4 = fabric_slice.add_node(
                    name=h4_name,
                    site=h4_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h4_iface = h4.add_component(model='NIC_ConnectX_6', name="h4_nic").get_interfaces()[0]

    #Add host networks 
    host_net_h1_r1 = fabric_slice.add_l2network(name=net_h1_r1_name, interfaces=[r1_iface_to_h1, h1_iface])
    host_net_h2_r1 = fabric_slice.add_l2network(name=net_h2_r1_name, interfaces=[r1_iface_to_h2, h2_iface])
    host_net_h3_r2 = fabric_slice.add_l2network(name=net_h3_r2_name, interfaces=[r2_iface_to_h3, h3_iface])
    host_net_h4_r2 = fabric_slice.add_l2network(name=net_h4_r2_name, interfaces=[r2_iface_to_h4, h4_iface])
    host_net_r1_r2 = fabric_slice.add_l2network(name=net_r1_r2_name, interfaces=[r1_iface_to_r2, r2_iface_to_r1])
    
    #Submit slice Request
    fabric_slice.submit(wait=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


Retry: 6, Time: 445 sec


ID,061af533-e324-4495-b303-8952d01dd005
Name,PoSeiDon-TCP-Experiment-ConnectX5_6
Lease Expiration (UTC),2023-03-28 01:32:38 +0000
Lease Start (UTC),2023-03-27 01:32:40 +0000
Project ID,1a0f06eb-75ea-4258-9b92-ff4d8cf999b0
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
bf79404d-0c43-4f3c-ac90-fbd65e095821,poseidon-router-1,24,32,100,default_ubuntu_20,qcow2,wash-w2.fabric-testbed.net,WASH,ubuntu,2001:400:a100:3020:f816:3eff:fe66:bb07,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/keys/fabric_sliver_key.pub,/home/fabric/work/fabric_config/keys/fabric_sliver_key
98ef27b0-fece-4055-9835-a2e391a80425,poseidon-router-2,24,32,100,default_ubuntu_20,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe0c:3f78,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/keys/fabric_sliver_key.pub,/home/fabric/work/fabric_config/keys/fabric_sliver_key
be6a80d3-715c-4f85-b362-73aeb39f59f9,poseidon-sender-1,26,32,100,default_ubuntu_20,qcow2,clem-w3.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fe97:559e,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/keys/fabric_sliver_key.pub,/home/fabric/work/fabric_config/keys/fabric_sliver_key
b9716168-0615-49c6-a6cf-07c30caab1e7,poseidon-sender-2,26,32,100,default_ubuntu_20,qcow2,clem-w3.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fecb:ce87,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/keys/fabric_sliver_key.pub,/home/fabric/work/fabric_config/keys/fabric_sliver_key
6db69757-0504-4506-8960-3caa991c148d,poseidon-receiver-1,26,32,100,default_ubuntu_20,qcow2,tacc-w5.fabric-testbed.net,TACC,ubuntu,129.114.110.88,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/keys/fabric_sliver_key.pub,/home/fabric/work/fabric_config/keys/fabric_sliver_key
c55e203f-672c-4885-bb80-510892d3c687,poseidon-receiver-2,26,32,100,default_ubuntu_20,qcow2,tacc-w4.fabric-testbed.net,TACC,ubuntu,129.114.110.109,Active,,ssh ${Username}@${Management IP},/home/fabric/work/fabric_config/keys/fabric_sliver_key.pub,/home/fabric/work/fabric_config/keys/fabric_sliver_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
e95d37ce-9c16-4aaa-b75d-1b4cc0b3fed3,net_h1_r1,L2,L2STS,None,None,None,Active,
a1bb2125-d143-4598-a0e7-4b3a4baace41,net_h2_r1,L2,L2STS,None,None,None,Active,
607c4c8e-6bc6-4cef-b64b-771fec2f4f14,net_h3_r2,L2,L2STS,None,None,None,Active,
394338d3-d6fe-4276-b972-0aed2b2a55bc,net_h4_r2,L2,L2STS,None,None,None,Active,
6b09ee2e-b0e9-4b2a-a949-e101c79e986b,net_r1_r2,L2,L2STS,None,None,None,Active,



Time to stable 445 seconds
Running post_boot_config ... Time to post boot config 504 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
poseidon-router-1-r1_r2_nic-p1,poseidon-router-1,net_r1_r2,100,None,B8:CE:F6:96:E1:62,ens7,ens7
poseidon-router-1-r1_r2_nic-p2,poseidon-router-1,None,100,None,B8:CE:F6:96:E1:63,ens8,ens8
poseidon-router-1-r1_h1_h2_nic-p1,poseidon-router-1,net_h1_r1,100,None,B8:CE:F6:96:E1:6A,ens9,ens9
poseidon-router-1-r1_h1_h2_nic-p2,poseidon-router-1,net_h2_r1,100,None,B8:CE:F6:96:E1:6B,ens10,ens10
poseidon-router-2-r2_r1_nic-p1,poseidon-router-2,net_r1_r2,100,None,B8:CE:F6:37:76:0A,ens7,ens7
poseidon-router-2-r2_r1_nic-p2,poseidon-router-2,None,100,None,B8:CE:F6:37:76:0B,ens8,ens8
poseidon-router-2-r2_h3_h4_nic-p2,poseidon-router-2,net_h3_r2,100,None,B8:CE:F6:37:75:1B,ens10,ens10
poseidon-router-2-r2_h3_h4_nic-p1,poseidon-router-2,net_h4_r2,100,None,B8:CE:F6:37:75:1A,ens9,ens9
poseidon-sender-1-h1_nic-p1,poseidon-sender-1,net_h1_r1,25,None,10:70:FD:C0:EF:70,ens7,ens7
poseidon-sender-1-h1_nic-p2,poseidon-sender-1,None,25,None,10:70:FD:C0:EF:71,ens8,ens8



Time to print interfaces 533 seconds


## Upload Post Boot Script

In [5]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for node in fabric_slice.get_nodes():
        node_site = node.get_site()
        if node_site in ["MAX", "MASS", "TACC"]:
            node.upload_file("setup-ipv4.sh", "/home/ubuntu/setup.sh")
        else:
            node.upload_file("setup-ipv6.sh", "/home/ubuntu/setup.sh")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Install Post Boot Script

In [6]:
try:
    config_command = "sudo bash /home/ubuntu/setup.sh"
    
    #Create execute threads
    execute_threads = {}
    for node in fabric_slice.get_nodes():
        print(f"Starting config on node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()

    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

Starting config on node poseidon-router-1
Starting config on node poseidon-router-2
Starting config on node poseidon-sender-1
Starting config on node poseidon-sender-2
Starting config on node poseidon-receiver-1
Starting config on node poseidon-receiver-2
Waiting for result from node poseidon-router-1
Waiting for result from node poseidon-router-2
Waiting for result from node poseidon-sender-1
Waiting for result from node poseidon-sender-2
Waiting for result from node poseidon-receiver-1
Waiting for result from node poseidon-receiver-2
All Done!


## Reboot Nodes To Load New Kernel

In [7]:
try:
    config_command = "sudo reboot"
    
    #Create execute threads
    execute_threads = {}
    for node in fabric_slice.get_nodes():
        print(f"Rebooting node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()

    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

Rebooting node poseidon-router-1
Rebooting node poseidon-router-2
Rebooting node poseidon-sender-1
Rebooting node poseidon-sender-2
Rebooting node poseidon-receiver-1
Rebooting node poseidon-receiver-2
Waiting for result from node poseidon-router-1


Exception in thread Thread-623:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/fabric/.local/lib/python3.9/site-packages/paramiko/transport.py", line 2211, in run
    self.packetizer.close()
  File "/home/fabric/.local/lib/python3.9/site-packages/paramiko/packet.py", line 207, in close
    self.__socket.close()
  File "/home/fabric/.local/lib/python3.9/site-packages/paramiko/channel.py", line 669, in close
    self.transport._send_user_message(m)
  File "/home/fabric/.local/lib/python3.9/site-packages/paramiko/transport.py", line 1915, in _send_user_message
    self._send_message(data)
  File "/home/fabric/.local/lib/python3.9/site-packages/paramiko/transport.py", line 1891, in _send_message
    self.packetizer.send_message(data)
  File "/home/fabric/.local/lib/python3.9/site-packages/paramiko/packet.py", line 425, in send_message
    self.write_all(out)
  File "/home/fabric/.local/lib/python

Waiting for result from node poseidon-router-2
Waiting for result from node poseidon-sender-1
Waiting for result from node poseidon-sender-2
Waiting for result from node poseidon-receiver-1
Waiting for result from node poseidon-receiver-2
All Done!


In [8]:
try:
    config_command = "ls -al"
    
    #Check if nodes are up
    for node in fabric_slice.get_nodes():
        print(f"Waiting for node {node.get_name()}")
        stdout,stderr = node.execute(config_command)
        print(stdout)
        print(stderr)
        
    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

Waiting for node poseidon-router-1
total 2163116
drwxr-xr-x 5 ubuntu ubuntu       4096 Mar 27 01:47 .
drwxr-xr-x 3 root   root         4096 Mar 27 01:34 ..
drwx------ 3 ubuntu ubuntu       4096 Mar 27 01:34 .ansible
-rw-r--r-- 1 ubuntu ubuntu        220 Feb 25  2020 .bash_logout
-rw-r--r-- 1 ubuntu ubuntu       3771 Feb 25  2020 .bashrc
drwx------ 2 ubuntu ubuntu       4096 Mar 27 01:34 .cache
-rw-r--r-- 1 ubuntu ubuntu        807 Feb 25  2020 .profile
drwx------ 2 ubuntu ubuntu       4096 Mar 27 01:34 .ssh
-rw-r--r-- 1 ubuntu ubuntu          0 Mar 27 01:34 .sudo_as_admin_successful
-rw-r--r-- 1 root   root   2214987211 Dec  5 21:38 kernel-+v2alpha+a23c4bb59e0c+FABRIC.tar.gz2
-rw-rw-r-- 1 ubuntu ubuntu       1627 Mar 27 01:44 setup.sh
total 2163116
drwxr-xr-x 5 ubuntu ubuntu       4096 Mar 27 01:47 .
drwxr-xr-x 3 root   root         4096 Mar 27 01:34 ..
drwx------ 3 ubuntu ubuntu       4096 Mar 27 01:34 .ansible
-rw-r--r-- 1 ubuntu ubuntu        220 Feb 25  2020 .bash_logout
-rw-r--r--

## Configure Poseidon Router 1

In [9]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    r1 = fabric_slice.get_node(name=r1_name)
    
    stdout1, stderr = r1.execute('sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
        
    r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
    r1_r2_os_iface = r1_iface_r2.get_os_interface()
    stdout, stderr = r1.execute(f'sudo ip link set dev {r1_r2_os_iface} up', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip link set mtu 8900 dev {r1_r2_os_iface}', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip addr add 192.168.1.1/30 dev {r1_r2_os_iface}', quiet=True)
    #stdout, stderr = r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root fq', quiet=True)
    
    r1_iface_h1 = r1.get_interface(network_name=net_h1_r1_name)
    r1_h1_os_iface = r1_iface_h1.get_os_interface()
    stdout, stderr = r1.execute(f'sudo ip link set dev {r1_h1_os_iface} up', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip link set mtu 8900 dev {r1_h1_os_iface}', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip addr add 192.168.2.1/30 dev {r1_h1_os_iface}', quiet=True)
    #stdout, stderr = r1.execute(f'sudo tc qdisc add dev {r1_h1_os_iface} root fq', quiet=True)
    
    r1_iface_h2 = r1.get_interface(network_name=net_h2_r1_name)
    r1_h2_os_iface = r1_iface_h2.get_os_interface()
    stdout, stderr = r1.execute(f'sudo ip link set dev {r1_h2_os_iface} up', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip link set mtu 8900 dev {r1_h2_os_iface}', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip addr add 192.168.3.1/30 dev {r1_h2_os_iface}', quiet=True)
    #stdout, stderr = r1.execute(f'sudo tc qdisc add dev {r1_h2_os_iface} root fq', quiet=True)
    
    stdout, stderr = r1.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.1.2', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.1.2', quiet=True)
    
    stdout, stderr = r1.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = r1.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

default via 10.20.4.1 dev ens3 proto dhcp src 10.20.5.146 metric 100 
10.20.4.0/23 dev ens3 proto kernel scope link src 10.20.5.146 
169.254.169.254 via 10.20.4.11 dev ens3 proto dhcp src 10.20.5.146 metric 100 
192.168.1.0/30 dev ens7np0 proto kernel scope link src 192.168.1.1 
192.168.2.0/30 dev ens9np0 proto kernel scope link src 192.168.2.1 
192.168.3.0/30 dev ens10np0 proto kernel scope link src 192.168.3.1 
192.168.4.0/30 via 192.168.1.2 dev ens7np0 
192.168.5.0/30 via 192.168.1.2 dev ens7np0 

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:66:bb:07 brd ff:ff:ff:ff:ff:ff
    altname enp0s3
    inet 10

## Configure Poseidon Router 2

In [10]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    r2 = fabric_slice.get_node(name=r2_name)
    
    stdout1, stderr = r2.execute('sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
        
    r2_iface_r1 = r2.get_interface(network_name=net_r1_r2_name)    
    r2_r1_os_iface = r2_iface_r1.get_os_interface()
    stdout, stderr = r2.execute(f'sudo ip link set dev {r2_r1_os_iface} up', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip link set mtu 8900 dev {r2_r1_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip addr add 192.168.1.2/30 dev {r2_r1_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo tc qdisc add dev {r2_r1_os_iface} root fq', quiet=True)
    
    r2_iface_h3 = r2.get_interface(network_name=net_h3_r2_name)
    r2_h3_os_iface = r2_iface_h3.get_os_interface()
    stdout, stderr = r2.execute(f'sudo ip link set dev {r2_h3_os_iface} up', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip link set mtu 8900 dev {r2_h3_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip addr add 192.168.4.1/30 dev {r2_h3_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo tc qdisc add dev {r2_h3_os_iface} root fq', quiet=True)
    
    r2_iface_h4 = r2.get_interface(network_name=net_h4_r2_name)
    r2_h4_os_iface = r2_iface_h4.get_os_interface()
    stdout, stderr = r2.execute(f'sudo ip link set dev {r2_h4_os_iface} up', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip link set mtu 8900 dev {r2_h4_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip addr add 192.168.5.1/30 dev {r2_h4_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo tc qdisc add dev {r2_h4_os_iface} root fq', quiet=True)

    stdout, stderr = r2.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.1.1', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.1.1', quiet=True)
    
    stdout, stderr = r2.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = r2.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

10.20.4.0/23 dev ens3 proto kernel scope link src 10.20.5.107 
169.254.169.254 via 10.20.4.11 dev ens3 proto dhcp src 10.20.5.107 metric 100 
192.168.1.0/30 dev ens7np0 proto kernel scope link src 192.168.1.2 
192.168.2.0/30 via 192.168.1.1 dev ens7np0 
192.168.3.0/30 via 192.168.1.1 dev ens7np0 
192.168.4.0/30 dev ens10np0 proto kernel scope link src 192.168.4.1 
192.168.5.0/30 dev ens9np0 proto kernel scope link src 192.168.5.1 

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:0c:3f:78 brd ff:ff:ff:ff:ff:ff
    altname enp0s3
    inet 10.20.5.107/23 brd 10.20.5.255 scope global dynamic ens3
       valid_lf

## Configure Poseidon Host 1

In [20]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host1 = fabric_slice.get_node(name=h1_name)
    
    host1_iface = host1.get_interface(network_name=net_h1_r1_name)
    
    host1_os_iface = host1_iface.get_os_interface()

    stdout, stderr = host1.execute(f'sudo ip link set dev {host1_os_iface} up', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip link set mtu 8900 dev {host1_os_iface}', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip addr add 192.168.2.2/30 dev {host1_os_iface}', quiet=True)
    stdout, stderr = host1.execute(f'sudo tc qdisc add dev {host1_os_iface} root fq', quiet=True)
    
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.2.1', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.2.1', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.2.1', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.2.1', quiet=True)
    
    stdout, stderr = host1.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host1.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

default via 10.30.6.1 dev ens3 proto dhcp src 10.30.6.168 metric 100 
10.30.6.0/23 dev ens3 proto kernel scope link src 10.30.6.168 
169.254.169.254 via 10.30.6.11 dev ens3 proto dhcp src 10.30.6.168 metric 100 
192.168.1.0/30 via 192.168.2.1 dev ens7np0 
192.168.2.0/30 dev ens7np0 proto kernel scope link src 192.168.2.2 
192.168.3.0/30 via 192.168.2.1 dev ens7np0 
192.168.4.0/30 via 192.168.2.1 dev ens7np0 
192.168.5.0/30 via 192.168.2.1 dev ens7np0 

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:97:55:9e brd ff:ff:ff:ff:ff:ff
    altname enp0s3
    inet 10.30.6.168/23 brd 10.30.7.255 scope global dynamic

## Configure Poseidon Host 2

In [21]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host2 = fabric_slice.get_node(name=h2_name)
    
    host2_iface = host2.get_interface(network_name=net_h2_r1_name)  
    
    host2_os_iface = host2_iface.get_os_interface()
    
    stdout, stderr = host2.execute(f'sudo ip link set dev {host2_os_iface} up', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip link set mtu 8900 dev {host2_os_iface}', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip addr add 192.168.3.2/30 dev {host2_os_iface}', quiet=True)
    stdout, stderr = host2.execute(f'sudo tc qdisc add dev {host2_os_iface} root fq', quiet=True)
    
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.3.1', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.3.1', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.3.1', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.3.1', quiet=True)
    
    stdout, stderr = host2.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host2.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

default via 10.30.6.1 dev ens3 proto dhcp src 10.30.6.62 metric 100 
10.30.6.0/23 dev ens3 proto kernel scope link src 10.30.6.62 
169.254.169.254 via 10.30.6.11 dev ens3 proto dhcp src 10.30.6.62 metric 100 
192.168.1.0/30 via 192.168.3.1 dev ens7np0 
192.168.2.0/30 via 192.168.3.1 dev ens7np0 
192.168.3.0/30 dev ens7np0 proto kernel scope link src 192.168.3.2 
192.168.4.0/30 via 192.168.3.1 dev ens7np0 
192.168.5.0/30 via 192.168.3.1 dev ens7np0 

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:cb:ce:87 brd ff:ff:ff:ff:ff:ff
    altname enp0s3
    inet 10.30.6.62/23 brd 10.30.7.255 scope global dynamic ens

## Configure Poseidon Host 3

In [13]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host3 = fabric_slice.get_node(name=h3_name)
    
    host3_iface = host3.get_interface(network_name=net_h3_r2_name)
    
    host3_os_iface = host3_iface.get_os_interface()
    
    stdout, stderr = host3.execute(f'sudo ip link set dev {host3_os_iface} up', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip link set mtu 8900 dev {host3_os_iface}', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip addr add 192.168.4.2/30 dev {host3_os_iface}', quiet=True)
    stdout, stderr = host3.execute(f'sudo tc qdisc add dev {host3_os_iface} root fq', quiet=True)
    
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.4.1', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.4.1', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.4.1', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.4.1', quiet=True)
    
    stdout, stderr = host3.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host3.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

default via 10.20.4.1 dev ens3 proto dhcp src 10.20.5.248 metric 100 
10.20.4.0/23 dev ens3 proto kernel scope link src 10.20.5.248 
169.254.169.254 via 10.20.4.1 dev ens3 proto dhcp src 10.20.5.248 metric 100 
192.168.1.0/30 via 192.168.4.1 dev ens7np0 
192.168.2.0/30 via 192.168.4.1 dev ens7np0 
192.168.3.0/30 via 192.168.4.1 dev ens7np0 
192.168.4.0/30 dev ens7np0 proto kernel scope link src 192.168.4.2 
192.168.5.0/30 via 192.168.4.1 dev ens7np0 

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:5f:42:35 brd ff:ff:ff:ff:ff:ff
    altname enp0s3
    inet 10.20.5.248/23 brd 10.20.5.255 scope global dynamic 

## Configure Poseidon Host 4

In [14]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host4 = fabric_slice.get_node(name=h4_name)
    
    host4_iface = host4.get_interface(network_name=net_h4_r2_name)  
     
    host4_os_iface = host4_iface.get_os_interface()
    
    stdout, stderr = host4.execute(f'sudo ip link set dev {host4_os_iface} up', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip link set mtu 8900 dev {host4_os_iface}', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip addr add 192.168.5.2/30 dev {host4_os_iface}', quiet=True)
    stdout, stderr = host4.execute(f'sudo tc qdisc add dev {host4_os_iface} root fq', quiet=True)
    
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.5.1', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.5.1', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.5.1', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.5.1', quiet=True)
    
    stdout, stderr = host4.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host4.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

default via 10.20.4.1 dev ens3 proto dhcp src 10.20.4.75 metric 100 
10.20.4.0/23 dev ens3 proto kernel scope link src 10.20.4.75 
169.254.169.254 via 10.20.4.1 dev ens3 proto dhcp src 10.20.4.75 metric 100 
192.168.1.0/30 via 192.168.5.1 dev ens8np0 
192.168.2.0/30 via 192.168.5.1 dev ens8np0 
192.168.3.0/30 via 192.168.5.1 dev ens8np0 
192.168.4.0/30 via 192.168.5.1 dev ens8np0 
192.168.5.0/30 dev ens8np0 proto kernel scope link src 192.168.5.2 

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:ba:3d:94 brd ff:ff:ff:ff:ff:ff
    altname enp0s3
    inet 10.20.4.75/23 brd 10.20.5.255 scope global dynamic ens3

## Ping Router 1 from Router 2

In [29]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    router1 = fabric_slice.get_node(name=r1_name)
    stdout, stderr = router1.execute(f'ping -c 4 192.168.1.2', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

PING 192.168.1.2 (192.168.1.2) 56(84) bytes of data.
64 bytes from 192.168.1.2: icmp_seq=1 ttl=64 time=16.8 ms
64 bytes from 192.168.1.2: icmp_seq=2 ttl=64 time=16.6 ms
64 bytes from 192.168.1.2: icmp_seq=3 ttl=64 time=16.6 ms
64 bytes from 192.168.1.2: icmp_seq=4 ttl=64 time=16.6 ms

--- 192.168.1.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3005ms
rtt min/avg/max/mdev = 16.594/16.664/16.823/0.093 ms



## Ping Host 3 from Host 1

In [28]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host1 = fabric_slice.get_node(name=h1_name)
    stdout, stderr = host1.execute(f'ping -c 4 192.168.4.2', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

PING 192.168.4.2 (192.168.4.2) 56(84) bytes of data.
64 bytes from 192.168.4.2: icmp_seq=1 ttl=62 time=62.0 ms
64 bytes from 192.168.4.2: icmp_seq=2 ttl=62 time=61.2 ms
64 bytes from 192.168.4.2: icmp_seq=3 ttl=62 time=61.2 ms
64 bytes from 192.168.4.2: icmp_seq=4 ttl=62 time=61.3 ms

--- 192.168.4.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3004ms
rtt min/avg/max/mdev = 61.219/61.444/62.017/0.332 ms



## Ping Host 4 from Host 2

In [83]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host2 = fabric_slice.get_node(name=h2_name)
    stdout, stderr = host2.execute(f'ping -c 4 192.168.5.2', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

PING 192.168.5.2 (192.168.5.2) 56(84) bytes of data.
64 bytes from 192.168.5.2: icmp_seq=1 ttl=62 time=62.1 ms
64 bytes from 192.168.5.2: icmp_seq=2 ttl=62 time=61.4 ms
64 bytes from 192.168.5.2: icmp_seq=3 ttl=62 time=61.4 ms
64 bytes from 192.168.5.2: icmp_seq=4 ttl=62 time=61.3 ms

--- 192.168.5.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3004ms
rtt min/avg/max/mdev = 61.330/61.559/62.115/0.322 ms



## Upload Iperf3 and Gather Total Scripts to Host 1,2,3,4

In [18]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for node_name in [h1_name, h2_name]:
        node = fabric_slice.get_node(name=node_name)
        node.upload_file("start_iperf_client.sh", "/home/ubuntu/start_iperf_client.sh")
        node.upload_file("gather_total.sh", "/home/ubuntu/gather_total.sh")

    for node_name in [h3_name, h4_name]:
        node = fabric_slice.get_node(name=node_name)
        node.upload_file("start_iperf_server.sh", "/home/ubuntu/start_iperf_server.sh")

except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Extend Slice's Life

In [144]:
from pytz import timezone
import datetime
end_date = (datetime.datetime.now(timezone("America/Los_Angeles")) + datetime.timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    fabric_slice.renew(end_date)
    fabric_slice = fablib.get_slice(name=slice_name)
    print(f'New lease end time: {fabric_slice.get_lease_end()}')
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

New lease end time: 2023-05-10 05:03:37 +0000


## Speed and CCA configuration

In [134]:
mtu_setting = 8900
rtt_setting = 0.062 #change this based on the ping reponse from earlier cells

speed_config = [
    {"tag": "100mbps", "speed": 100000000, "processes": 1, "parallel_streams": 1},
    {"tag": "500mbps", "speed": 500000000, "processes": 5, "parallel_streams": 1},
    {"tag": "1gbps", "speed": 1000000000, "processes": 10, "parallel_streams": 1},
    {"tag": "10gbps", "speed": 10000000000, "processes": 20, "parallel_streams": 5},
    {"tag": "25gbps", "speed": 25000000000, "processes": 25, "parallel_streams": 10}
]

cca_config = [
    {"cca1": "bbr", "cca2": "cubic"},
    {"cca1": "bbr2", "cca2": "cubic"},
    {"cca1": "htcp", "cca2": "cubic"},
    {"cca1": "reno", "cca2": "cubic"},
    {"cca1": "cubic", "cca2": "cubic"},
    {"cca1": "bbr", "cca2": "bbr"},
    {"cca1": "bbr2", "cca2": "bbr2"},
    {"cca1": "htcp", "cca2": "htcp"},
    {"cca1": "reno", "cca2": "reno"}
]

## FIFO TESTS

In [ ]:
%%capture output

try:
    fabric_slice = fablib.get_slice(name=slice_name)
    
    for cca_conf in cca_config:
        cca1 = cca_conf["cca1"]
        cca2 = cca_conf["cca2"]
        
        for speed_conf in speed_config:
            speed = speed_conf["speed"]
            processes = speed_conf["processes"]
            streams = speed_conf["parallel_streams"]
            tag = speed_conf["tag"]

            bdp = speed * rtt_setting / 8.0 #this is in bytes
            
            for mult in [0.5,1,2,4,8,16]:
            #for mult in [16]:
                print(f"{cca1}-{cca2}-{tag}-fifo-{mult}bdp")
                
                applied_bdp = int(bdp * mult)
                packets = int(applied_bdp / mtu_setting) #mtu is 8900 bytes
                
                print(f"Configuring buffer on node {r1_name}")
                r1 = fabric_slice.get_node(name=r1_name)
        
                r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
                r1_r2_os_iface = r1_iface_r2.get_os_interface()

                #r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root')
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root netem limit {packets} rate {speed}')
                
                print(f"iface: {r1_r2_os_iface}, speed: {speed}, packets: {packets}")
                r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: tbf rate {speed} burst 1570000 limit {applied_bdp}', quiet=False)
                
                #run 5 reps
                for rep in range(1,6):
                    print(f"Starting run {rep}")
                    #start servers
                    print(f"Starting servers on node {h3_name}")
                    node = fabric_slice.get_node(name=h3_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca1} {cca2} fifo {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting servers on node {h4_name}")
                    node = fabric_slice.get_node(name=h4_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca2} {cca1} fifo {mult}bdp {rep} {processes}", quiet=True)

                    #if cca1 in ["htcp", "bbr2"]:
                    #    cmd_host_1 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} fifo {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    #else:
                    cmd_host_1 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} fifo {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    
                    #if cca2 in ["htcp", "bbr2"]:
                    #    cmd_host_2 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} fifo {mult}bdp {rep} {processes} 192.168.5.2 {streams}"
                    #else:
                    cmd_host_2 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} fifo {mult}bdp {rep} {processes} 192.168.5.2 {streams}"

                    #start clients via execute threads
                    execute_threads = {}
                    print(f"Starting clients on node {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    execute_threads[node] = node.execute_thread(cmd_host_1)
                    print(f"Starting clients on node {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    execute_threads[node] = node.execute_thread(cmd_host_2)
                   
                    #Wait for results from threads
                    for node,thread in execute_threads.items():
                        #print(f"Waiting for result from node {node.get_name()}")
                        stdout,stderr = thread.result()

                    #iperf runs for 200seconds so we wait for 210
                    print("Waiting for iperf3 to finish for 210seconds")
                    sleep(210)

                    #kill all iperf processes
                    kill_command = "pkill -f iperf3"

                    for node_name in [h1_name, h2_name, h3_name, h4_name]:
                        node = fabric_slice.get_node(name=node_name)
                        print(f"Killing iperf3 on node {node.get_name()}")
                        stdout,stderr = node.execute(kill_command)
    print("All done with fifo!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [125]:
output.show()

bbr-cubic-25gbps-fifo-0.5bdp
Configuring buffer on node poseidon-router-1
iface: ens7np0, speed: 25000000000, packets: 10884
 sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
Error: Cannot delete qdisc with handle of zero.
  sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
 Starting run 1
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node poseidon-sender-1
Starting clients on node poseidon-sender-2
Waiting for iperf3 to finish for 210seconds
Killing iperf3 on node poseidon-sender-1
Killing iperf3 on node poseidon-sender-2
Killing iperf3 on node poseidon-receiver-1
Killing iperf3 on node poseidon-receiver-2
Starting run 2
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node poseidon-sender-1
Starting clients on node poseidon-sender-2
Waiting

## FQ_CODEL TESTS

In [ ]:
%%capture output

try:
    fabric_slice = fablib.get_slice(name=slice_name)
    
    for cca_conf in cca_config:
        cca1 = cca_conf["cca1"]
        cca2 = cca_conf["cca2"]
        
        for speed_conf in speed_config:
            speed = speed_conf["speed"]
            processes = speed_conf["processes"]
            streams = speed_conf["parallel_streams"]
            tag = speed_conf["tag"]

            bdp = speed * rtt_setting / 8.0 #this is in bytes
            
            for mult in [0.5,1,2,4,8,16]:
                print(f"{cca1}-{cca2}-{tag}-fq_codel-{mult}bdp")
                
                applied_bdp = int(bdp * mult)
                packets = int(applied_bdp / mtu_setting) #mtu is 8900 bytes
                
                print(f"Configuring buffer on node {r1_name}")
                r1 = fabric_slice.get_node(name=r1_name)
        
                r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
                r1_r2_os_iface = r1_iface_r2.get_os_interface()

                print(f"iface: {r1_r2_os_iface}, speed: {speed}, packets: {packets}")
                r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: tbf rate {speed} burst 1570000 limit {applied_bdp}', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: fq_codel limit {packets}', quiet=False)
                
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1:0 netem limit 10 rate {speed}', quiet=False)
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: fq_codel limit {packets}', quiet=False)
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: htb default 1', quiet=False)
                #r1.execute(f'sudo tc class add dev {r1_r2_os_iface} parent 1: classid 1:1 htb rate {speed}', quiet=False)
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 fq_codel limit {packets}', quiet=False)
                
                
                #run 5 reps
                for rep in range(1,6):
                    print(f"Starting run {rep}")
                    #start servers
                    print(f"Starting servers on node {h3_name}")
                    node = fabric_slice.get_node(name=h3_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca1} {cca2} fq_codel {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting servers on node {h4_name}")
                    node = fabric_slice.get_node(name=h4_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca2} {cca1} fq_codel {mult}bdp {rep} {processes}", quiet=True)

                    #if cca1 in ["htcp", "bbr2"]:
                    #    cmd_host_1 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} fq_codel {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    #else:
                    cmd_host_1 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} fq_codel {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    
                    #if cca2 in ["htcp", "bbr2"]:
                    #    cmd_host_2 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} fq_codel {mult}bdp {rep} {processes} 192.168.5.2 {streams}"
                    #else:
                    cmd_host_2 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} fq_codel {mult}bdp {rep} {processes} 192.168.5.2 {streams}"

                    #start clients via execute threads
                    execute_threads = {}
                    print(f"Starting clients on node {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    execute_threads[node] = node.execute_thread(cmd_host_1)
                    print(f"Starting clients on node {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    execute_threads[node] = node.execute_thread(cmd_host_2)

                    #Wait for results from threads
                    for node,thread in execute_threads.items():
                        #print(f"Waiting for result from node {node.get_name()}")
                        stdout,stderr = thread.result()

                    #iperf runs for 200seconds so we wait for 210
                    print("Waiting for iperf3 to finish for 210seconds")
                    sleep(210)

                    #kill all iperf processes
                    kill_command = "pkill -f iperf3"

                    for node_name in [h1_name, h2_name, h3_name, h4_name]:
                        node = fabric_slice.get_node(name=node_name)
                        print(f"Killing iperf3 on node {node.get_name()}")
                        stdout,stderr = node.execute(kill_command)
    print("All done with fq_codel!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [128]:
output.show()

bbr-cubic-25gbps-fq_codel-0.5bdp
Configuring buffer on node poseidon-router-1
iface: ens7np0, speed: 25000000000, packets: 10884
 sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
  sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
  sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
 Starting run 1
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node poseidon-sender-1
Starting clients on node poseidon-sender-2
Waiting for iperf3 to finish for 210seconds
Killing iperf3 on node poseidon-sender-1
Killing iperf3 on node poseidon-sender-2
Killing iperf3 on node poseidon-receiver-1
Killing iperf3 on node poseidon-receiver-2
Starting run 2
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node

In [70]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    #kill all iperf processes
    kill_command = "pkill -f iperf3"

    for node_name in [h1_name, h2_name, h3_name, h4_name]:
        node = fabric_slice.get_node(name=node_name)
        print(f"Killing iperf3 on node {node.get_name()}")
        stdout,stderr = node.execute(kill_command)
    print("All done!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Killing iperf3 on node poseidon-sender-1
Killing iperf3 on node poseidon-sender-2
Killing iperf3 on node poseidon-receiver-1
Killing iperf3 on node poseidon-receiver-2
All done!


## RED TESTS

In [ ]:
%%capture output

try:
    fabric_slice = fablib.get_slice(name=slice_name)
    
    for cca_conf in cca_config:
        cca1 = cca_conf["cca1"]
        cca2 = cca_conf["cca2"]
        
        for speed_conf in speed_config:
            speed = speed_conf["speed"]
            processes = speed_conf["processes"]
            streams = speed_conf["parallel_streams"]
            tag = speed_conf["tag"]

            bdp = speed * rtt_setting / 8.0 #this is in bytes
            
            for mult in [0.5,1,2,4,8,16]:
            #for mult in [2,16]:
                print(f"{cca1}-{cca2}-{tag}-red-{mult}bdp")
                
                applied_bdp = int(bdp * mult)
                packets = int(applied_bdp / mtu_setting) #mtu is 8900 bytes
                
                print(f"Configuring buffer on node {r1_name}")
                r1 = fabric_slice.get_node(name=r1_name)
        
                r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
                r1_r2_os_iface = r1_iface_r2.get_os_interface()

                print(f"iface: {r1_r2_os_iface}, speed: {speed}, packets: {packets}")
                r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: tbf rate {speed} burst 1570000 limit {applied_bdp}', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: red limit {applied_bdp} avpkt 1000 min 30000 max 30001 ecn adaptive bandwidth {speed}', quiet=False)

                #r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=True)
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1:0 netem limit 10 rate {speed}', quiet=True)
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: red limit {applied_bdp} avpkt 1000 min 30000 max 30001 ecn adaptive bandwidth {speed}', quiet=True)

                
                #run 5 reps
                for rep in range(1,6):
                    print(f"Starting run {rep}")
                    #start servers
                    print(f"Starting servers on node {h3_name}")
                    node = fabric_slice.get_node(name=h3_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca1} {cca2} red {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting servers on node {h4_name}")
                    node = fabric_slice.get_node(name=h4_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca2} {cca1} red {mult}bdp {rep} {processes}", quiet=True)

                    #if cca1 in ["htcp", "bbr2"]:
                    #    cmd_host_1 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} red {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    #else:
                    cmd_host_1 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} red {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    
                    #if cca2 in ["htcp", "bbr2"]:
                    #    cmd_host_2 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} red {mult}bdp {rep} {processes} 192.168.5.2 {streams}"
                    #else:
                    cmd_host_2 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} red {mult}bdp {rep} {processes} 192.168.5.2 {streams}"

                    #start clients via execute threads
                    execute_threads = {}
                    print(f"Starting clients on node {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    execute_threads[node] = node.execute_thread(cmd_host_1)
                    print(f"Starting clients on node {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    execute_threads[node] = node.execute_thread(cmd_host_2)

                    #Wait for results from threads
                    for node,thread in execute_threads.items():
                        #print(f"Waiting for result from node {node.get_name()}")
                        stdout,stderr = thread.result()

                    #iperf runs for 200seconds so we wait for 210
                    print("Waiting for iperf3 to finish for 210seconds")
                    sleep(210)

                    #kill all iperf processes
                    kill_command = "pkill -f iperf3"

                    for node_name in [h1_name, h2_name, h3_name, h4_name]:
                        node = fabric_slice.get_node(name=node_name)
                        print(f"Killing iperf3 on node {node.get_name()}")
                        stdout,stderr = node.execute(kill_command)
    print("All done with red!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [131]:
output.show()

bbr-cubic-25gbps-red-0.5bdp
Configuring buffer on node poseidon-router-1
iface: ens7np0, speed: 25000000000, packets: 10884
 sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
  sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
  sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
 Starting run 1
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node poseidon-sender-1
Starting clients on node poseidon-sender-2
Waiting for iperf3 to finish for 210seconds
Killing iperf3 on node poseidon-sender-1
Killing iperf3 on node poseidon-sender-2
Killing iperf3 on node poseidon-receiver-1
Killing iperf3 on node poseidon-receiver-2
Starting run 2
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node pose

## RED ECN OFF TESTS

In [ ]:
%%capture output

try:
    fabric_slice = fablib.get_slice(name=slice_name)
    
    for cca_conf in cca_config:
        cca1 = cca_conf["cca1"]
        cca2 = cca_conf["cca2"]
        
        for speed_conf in speed_config:
            speed = speed_conf["speed"]
            processes = speed_conf["processes"]
            streams = speed_conf["parallel_streams"]
            tag = speed_conf["tag"]

            bdp = speed * rtt_setting / 8.0 #this is in bytes
            
            for mult in [0.5,1,2,4,8,16]:
            #for mult in [2,16]:
                print(f"{cca1}-{cca2}-{tag}-red-{mult}bdp")
                
                applied_bdp = int(bdp * mult)
                packets = int(applied_bdp / mtu_setting) #mtu is 8900 bytes
                
                print(f"Configuring buffer on node {r1_name}")
                r1 = fabric_slice.get_node(name=r1_name)
        
                r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
                r1_r2_os_iface = r1_iface_r2.get_os_interface()

                print(f"iface: {r1_r2_os_iface}, speed: {speed}, packets: {packets}")
                r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: tbf rate {speed} burst 1570000 limit {applied_bdp}', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: red limit {applied_bdp} avpkt 1000 min 30000 max 30001 bandwidth {speed}', quiet=False)

                #r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=True)
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1:0 netem limit 10 rate {speed}', quiet=True)
                #r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: red limit {applied_bdp} avpkt 1000 min 30000 max 30001 ecn adaptive bandwidth {speed}', quiet=True)

                
                #run 5 reps
                for rep in range(1,6):
                    print(f"Starting run {rep}")
                    #start servers
                    print(f"Starting servers on node {h3_name}")
                    node = fabric_slice.get_node(name=h3_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca1} {cca2} red_noecn {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting servers on node {h4_name}")
                    node = fabric_slice.get_node(name=h4_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca2} {cca1} red_noecn {mult}bdp {rep} {processes}", quiet=True)

                    #if cca1 in ["htcp", "bbr2"]:
                    #    cmd_host_1 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} red {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    #else:
                    cmd_host_1 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} red_noecn {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    
                    #if cca2 in ["htcp", "bbr2"]:
                    #    cmd_host_2 = f"sudo bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} red {mult}bdp {rep} {processes} 192.168.5.2 {streams}"
                    #else:
                    cmd_host_2 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} red_noecn {mult}bdp {rep} {processes} 192.168.5.2 {streams}"

                    #start clients via execute threads
                    execute_threads = {}
                    print(f"Starting clients on node {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    execute_threads[node] = node.execute_thread(cmd_host_1)
                    print(f"Starting clients on node {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    execute_threads[node] = node.execute_thread(cmd_host_2)

                    #Wait for results from threads
                    for node,thread in execute_threads.items():
                        #print(f"Waiting for result from node {node.get_name()}")
                        stdout,stderr = thread.result()

                    #iperf runs for 200seconds so we wait for 210
                    print("Waiting for iperf3 to finish for 210seconds")
                    sleep(210)

                    #kill all iperf processes
                    kill_command = "pkill -f iperf3"

                    for node_name in [h1_name, h2_name, h3_name, h4_name]:
                        node = fabric_slice.get_node(name=node_name)
                        print(f"Killing iperf3 on node {node.get_name()}")
                        stdout,stderr = node.execute(kill_command)
    print("All done with red!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [136]:
output.show()

bbr-cubic-100mbps-red-0.5bdp
Configuring buffer on node poseidon-router-1
iface: ens7np0, speed: 100000000, packets: 43
 sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
  sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
  sudo: unable to resolve host bf79404d-0c43-4f3c-ac90-fbd65e095821-poseidon-router-1: Name or service not known
 Starting run 1
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node poseidon-sender-1
Starting clients on node poseidon-sender-2
Waiting for iperf3 to finish for 210seconds
Killing iperf3 on node poseidon-sender-1
Killing iperf3 on node poseidon-sender-2
Killing iperf3 on node poseidon-receiver-1
Killing iperf3 on node poseidon-receiver-2
Starting run 2
Starting servers on node poseidon-receiver-1
Starting servers on node poseidon-receiver-2
Starting clients on node poseidon

## Calculate Totals

In [ ]:
try:
    calculate_totals = f"bash /home/ubuntu/gather_total.sh"
   
    #Create execute threads
    execute_threads = {}
    node = fabric_slice.get_node(name=h1_name)
    execute_threads[node] = node.execute_thread(calculate_totals)
    node = fabric_slice.get_node(name=h2_name)
    execute_threads[node] = node.execute_thread(calculate_totals)
    
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()

    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

## Compress and download output logs

In [137]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for node_name in [h1_name, h2_name, h3_name, h4_name]:
        tar_command = f"tar -czvf {node_name}_output.tar.gz output"
        node = fabric_slice.get_node(name=node_name)
        print(f"Compressing output on node {node.get_name()}")
        stdout,stderr = node.execute(tar_command, quiet=True)
        print(f"Downloading compressed output from node {node.get_name()}")
        node.download_file(f"{node_name}_output.tar.gz", f"{node_name}_output.tar.gz")
    print("Done!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Compressing output on node poseidon-sender-1
Compressing output on node poseidon-sender-2
Compressing output on node poseidon-receiver-1
Compressing output on node poseidon-receiver-2
Done!


## Helpers

In [ ]:
try:
    kill_command = "pkill -f iperf3"
    
    fabric_slice = fablib.get_slice(name=slice_name)
    for node_name in [h1_name, h2_name, h3_name, h4_name]:
        node = fabric_slice.get_node(name=node_name)
        print(f"Killing iperf3 on node {node.get_name()}")
        stdout,stderr = node.execute(kill_command)
    print("Done!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
from time import sleep

try:
    #start servers
    node = fabric_slice.get_node(name=h4_name)
    node.execute(f"ps -ax | grep iperf3")

    node = fabric_slice.get_node(name=h3_name)
    node.execute(f"ps -ax | grep iperf3")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Connect to Slice Nodes

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for node in fabric_slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
#        print(f"   Interfaces        :  ")
#        for interface in node.get_interfaces():
#            print(f"       Name                : {interface.get_name()}")
#            print(f"           Bandwidth           : {interface.get_bandwidth()}")
#            print(f"           VLAN                : {interface.get_vlan()}")  
#            print(f"           MAC                 : {interface.get_mac()}") 
#            print(f"           OS iface name       : {interface.get_os_interface()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()


## Delete Slice

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    fabric_slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()